### Feature Learning for Handwritten Digits using Autoencoders in PyTorch

As the first deep learning project, we are going to perform feature learning using an autoencoder network for handwritten digits. Deep network implementations are differ from typical shallow networks; we will see some of the differences in this project. 

As the deep learning framework, we chose [PyTorch](https://pytorch.org/) as it is easy for beginners. Installation guide is available [here](https://pytorch.org/get-started/locally/). In this implementation, our dataset is the [scikit-learn handwritten digits](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_digits.html).

In [1]:
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader

from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
import numpy as np
import time

from utils import plot_loss, plot_images

#### Constants
One of the changes in deep learning implementations is the mini-batch processing, in which dataset is divided into mini-batches containing 16, 32, 64, or higher number of samples. Then mini-batch gradient descent is applied to handle the big data processing. Here, `BS` is the batch size. 

Moreover, `EPOCHs` shows the number of times the dataset is scanned in training phase. `LR` is also the learning rate.  

In [2]:
BS = 512
EPOCHS = 300
LR = 1e-3

#### 1. Load dataset

In [3]:
X, y = load_digits(return_X_y=True)
print(np.shape(X))

(1797, 64)


#### Train / Test & Mini-batch Splitting

the `train_test_split` function from `scikit-learn` package helps in the train / test splitting. Then, we use **[Dataloader class](https://pytorch.org/docs/stable/data.html)** from `pytorch` for mini-batch processing on train set. The *Dataloader class* provides a python **iterable**, which is an object capable of returning its members one at a time.

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=20)

print(np.shape(X_train))
print(np.shape(X_test))

trainloader = DataLoader(X_train, batch_size=BS, shuffle=True)

(1777, 64)
(20, 64)


#### 2. Network Architecture

The vanilla autoencoder contains **encoder** and **decoder** blocks. Our desired architecture has two layers in each block. We aim to use the undercomplete architecture, in which the bottleneck layer has less number of units compared to the input layer. Activation functions are ReLU as the input is density of image pixels that are numbers from 0 to 16.  

In followings, we define a class named `Autoencoder`, inherited from **[nn.Module class](https://pytorch.org/docs/master/generated/torch.nn.Module.html)** provided by pytorch for deep network architectures.  

In [9]:
class Autoencoder(nn.Module):
    def __init__(self):
        super(Autoencoder, self).__init__()
        
        #encoder
        self.enc1 = nn.Linear(in_features=64, out_features=32)
        self.enc2 = nn.Linear(in_features=32, out_features=10)
        #decoder
        self.dec1 = nn.Linear(in_features=10, out_features=32)
        self.dec2 = nn.Linear(in_features=32, out_features=64)
    
    def forward(self, x):
        x = F.relu(self.enc1(x))
        x = F.relu(self.enc2(x))
        x = F.relu(self.dec1(x))
        x = F.relu(self.dec2(x))
        return x

    def features(self, x):
        x = F.relu(self.enc1(x))
        x = F.relu(self.enc2(x))
        return x

net = Autoencoder()
print(net)

Autoencoder(
  (enc1): Linear(in_features=64, out_features=32, bias=True)
  (enc2): Linear(in_features=32, out_features=10, bias=True)
  (dec1): Linear(in_features=10, out_features=32, bias=True)
  (dec2): Linear(in_features=32, out_features=64, bias=True)
)


#### 3. Train
After *data preparation* and *network design*, the training step comes. For training, every machine learning project needs a **cost function** and an **optimization algorithm**. In this implementation, we chose [mean square error (MSE)](https://en.wikipedia.org/wiki/Mean_squared_error#:~:text=In%20statistics%2C%20the%20mean%20squared,values%20and%20the%20actual%20value.) as the cost function and [Adam optimizer](https://arxiv.org/abs/1412.6980). We utilize the **[nn.MSELoss class](https://pytorch.org/docs/master/generated/torch.nn.MSELoss.html)** and **[optim.Adam class](https://pytorch.org/docs/stable/optim.html#torch.optim.Adam)** provided by prtoch for each one, respectively.    

The main loop for training repeats for `EPOCHS` times. In every itertation, we calculate the loss and apply the optimizer on the trainable parameters in the network. Keeping the average loss values for each epoch, we plot the training loss curve at the end. 

In [15]:
criterion = nn.MSELoss()
optimizer = optim.Adam(net.parameters(), lr=LR)

tic = time.time()
train_loss = []
for epoch in range(EPOCHS):
    running_loss = 0.0
    for batch in trainloader:
        optimizer.zero_grad()
        outputs = net(batch.float())
        loss = criterion(outputs, batch.float())
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    loss = running_loss / len(trainloader)
    train_loss.append(loss)
tac = time.time()
print("Training is done in " + str(tac-tic) + " sec.""

print("last loss value: " + str(loss))
plot_loss(train_loss)

SyntaxError: EOL while scanning string literal (<ipython-input-15-258fbe947e98>, line 19)

#### 4. Test
Now, we can check how well the network can reconstruct test samples with the following code snippet.  

In [13]:
testloader = DataLoader(X_test, batch_size=20)

for batch in testloader:
    plot_images(batch)
    outputs = net(batch.float())
    plot_images(outputs.detach().numpy())